## STEP 9

In [48]:
from joblib import load

optimized_xgb = load('optimized_xgb_model.pkl')

y_pred = optimized_xgb.predict(X_test)

In [49]:
import numpy as np

unique, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique, counts)))


{0: 208280, 1: 8968}


In [50]:
print(df.columns)
df["Pct_afro_american"].head()
df["Pct_afro_american"].describe()

# the only ethcnicity variable is Pct_afro_american: Average percentage of African American people living in the area covered by the three-digit ZIP code (census)

Index(['issue_d', 'loan duration', 'annual_inc', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'delinq_2yrs', 'dti', 'emp_length',
       'emp_title', 'fico_range_high', 'funded_amnt', 'grade',
       'home_ownership', 'inq_last_6mths', 'int_rate', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'num_actv_bc_tl', 'num_bc_tl', 'num_il_tl',
       'num_rev_accts', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies',
       'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'target',
       'tax_liens', 'zip_code', 'Pct_afro_american'],
      dtype='object')


count    1.086236e+06
mean     1.290673e+01
std      1.206943e+01
min      4.301260e-02
25%      3.971440e+00
50%      8.868146e+00
75%      1.853919e+01
max      7.036799e+01
Name: Pct_afro_american, dtype: float64

In [51]:
#splitting the test set by median of this variable

# Extract protected attribute for test set
pct_afro = df.loc[X_test.index, "Pct_afro_american"]

# Median split
median_val = pct_afro.median()
ethnicity_group = pd.Series(
    ["High" if val > median_val else "Low" for val in pct_afro],
    index=pct_afro.index
)

print(ethnicity_group.value_counts())


Low     109416
High    107832
Name: count, dtype: int64


In [52]:
# Creating a new df : results_df
results_df = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred,
    "Pct_afro_american": df.loc[X_test.index, "Pct_afro_american"],
    "ethnicity_group": ethnicity_group,
    "annual_inc": df.loc[X_test.index, "annual_inc"],
    "fico_range_high": df.loc[X_test.index, "fico_range_high"],
    "grade": df.loc[X_test.index, "grade"],
    "loan_duration": df.loc[X_test.index, "loan duration"]   # rename to avoid space in column name
})

# ⚠️ Rename column so it has no space
results_df = results_df.rename(columns={"loan_duration": "loan_duration"})

# Quick check
print(results_df.head())
print(results_df["ethnicity_group"].value_counts())

         y_true  y_pred  Pct_afro_american ethnicity_group  annual_inc  \
199648      0.0       0           6.669588             Low    128000.0   
465152      0.0       0           4.657994             Low    128000.0   
1061172     0.0       0          15.194674            High     53000.0   
356750      0.0       0           1.698836             Low     31000.0   
485653      0.0       0           6.522108             Low     70000.0   

         fico_range_high grade  loan_duration  
199648             729.0     B            1.0  
465152             719.0     C            1.0  
1061172            779.0     A            0.0  
356750             674.0     D            0.0  
485653             674.0     D            0.0  
ethnicity_group
Low     109416
High    107832
Name: count, dtype: int64


In [53]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, roc_auc_score

# === 1. Statistical Parity (Demographic Parity)b
dp = results_df.groupby("ethnicity_group")["y_pred"].mean()
print("\nDemographic parity (selection rate):\n", dp)




Demographic parity (selection rate):
 ethnicity_group
High    0.046776
Low     0.035863
Name: y_pred, dtype: float64


In [54]:
# 2. Conditional statistical parity with safe ratio calculation
def conditional_stat_parity_stratified(
    df,
    group_col="ethnicity_group",
    cond_spec={"annual_inc": "q4", "fico_range_high": "q4", "grade": "cat", "loan_duration": "cat"},
    yhat_col="y_pred",
):
    work = df[[group_col, yhat_col] + list(cond_spec.keys())].copy()

    # Bin continuous variables, cast categoricals
    for col, spec in cond_spec.items():
        if spec.startswith("q"):  # e.g. "q4" for quartiles
            q = int(spec[1:])
            work[col] = pd.qcut(work[col], q=q, duplicates="drop")
        elif spec == "cat":
            work[col] = work[col].astype("category")
        else:
            raise ValueError(f"Unknown spec for {col}: {spec}")

    # Selection rate within each stratum and group
    strata = list(cond_spec.keys())
    rates = (
        work.groupby(strata + [group_col])[yhat_col]
            .mean()
            .unstack(group_col)
    )

    # Keep only strata where both groups are present
    rates = rates.dropna()

    # Per-stratum gap and ratio
    gaps = (rates.max(axis=1) - rates.min(axis=1)).rename("gap")
    ratios = (rates.min(axis=1) / rates.max(axis=1)).rename("ratio")

    # Stratum weights (number of samples per stratum)
    weights = work.groupby(strata).size().reindex(gaps.index).rename("n")

    # Weighted gap (always valid)
    w_gap = np.average(gaps.values, weights=weights.values)

    # Weighted ratio (only valid strata, drop NaNs)
    valid_ratios = ratios.dropna()
    if not valid_ratios.empty:
        w_ratio = np.average(valid_ratios.values, weights=weights.loc[valid_ratios.index].values)
    else:
        w_ratio = np.nan

    # Build summary table
    summary = pd.DataFrame({
        "n": weights,
        "gap": gaps,
        "ratio": ratios
    }).sort_values("n", ascending=False)

    return w_gap, w_ratio, rates, summary


# === Run it ===
w_gap, w_ratio, rate_table, per_stratum = conditional_stat_parity_stratified(
    results_df,
    group_col="ethnicity_group",
    cond_spec={
        "annual_inc": "q4",
        "fico_range_high": "q4",
        "grade": "cat",
        "loan_duration": "cat"
    },
    yhat_col="y_pred"
)

print("Weighted gap (max-min across groups within strata):", w_gap)
print("Weighted ratio (min/max across groups within strata, valid strata only):", w_ratio)



Weighted gap (max-min across groups within strata): 0.012221650699489016
Weighted ratio (min/max across groups within strata, valid strata only): 0.550842066648761


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/3551137818.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  work.groupby(strata + [group_col])[yhat_col]
/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/3551137818.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weights = work.groupby(strata).size().reindex(gaps.index).rename("n")


In [55]:

# === 3. Equal Opportunity (True Positive Rate per group)
def tpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return tp / (tp + fn) if (tp + fn) > 0 else float("nan")

tpr_values = results_df.groupby("ethnicity_group").apply(
    lambda g: tpr(g["y_true"], g["y_pred"])
)
print("\nTrue positive rate (Equal opportunity):\n", tpr_values)




True positive rate (Equal opportunity):
 ethnicity_group
High    0.123213
Low     0.098867
dtype: float64


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/15702600.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tpr_values = results_df.groupby("ethnicity_group").apply(


In [56]:
# === 4. False Positive Rate per group
def fpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return fp / (fp + tn) if (fp + tn) > 0 else float("nan")

fpr_values = results_df.groupby("ethnicity_group").apply(
    lambda g: fpr(g["y_true"], g["y_pred"])
)
print("\nFalse positive rate (Equalized odds):\n", fpr_values)




False positive rate (Equalized odds):
 ethnicity_group
High    0.025397
Low     0.019861
dtype: float64


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/390353249.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fpr_values = results_df.groupby("ethnicity_group").apply(


In [57]:
# === 5. Predictive Parity (Precision per group)
prec_values = results_df.groupby("ethnicity_group").apply(
    lambda g: precision_score(g["y_true"], g["y_pred"], zero_division=0)
)
print("\nPrecision (Predictive parity):\n", prec_values)


Precision (Predictive parity):
 ethnicity_group
High    0.575734
Low     0.558359
dtype: float64


/var/folders/jd/_41jr63j245210xpt44m7yjh0000gn/T/ipykernel_7075/2841586283.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prec_values = results_df.groupby("ethnicity_group").apply(


In [58]:
import pandas as pd

# Build summary DataFrame
fairness_summary = pd.DataFrame({
    "Statistical Parity (DP)": dp,
    "Equal Opportunity (TPR)": tpr_values,
    "Equalized Odds (FPR)": fpr_values,
    "Predictive Parity (Precision)": prec_values
})

# Compute gaps and ratios for each metric
gaps = fairness_summary.max() - fairness_summary.min()
ratios = fairness_summary.min() / fairness_summary.max()

# Nicely formatted output
print("=== Fairness Metrics by Group ===")
print(fairness_summary.round(4))
print("\n=== Disparities (across groups) ===")
print(pd.DataFrame({"Gap (max-min)": gaps.round(4), "Ratio (min/max)": ratios.round(3)}))

# Conditional SP
print("\n=== Conditional Statistical Parity ===")
print(f"Weighted gap (max-min across strata): {w_gap:.4f}")
print(f"Weighted ratio (valid strata only): {w_ratio:.3f}")


=== Fairness Metrics by Group ===
                 Statistical Parity (DP)  Equal Opportunity (TPR)  \
ethnicity_group                                                     
High                              0.0468                   0.1232   
Low                               0.0359                   0.0989   

                 Equalized Odds (FPR)  Predictive Parity (Precision)  
ethnicity_group                                                       
High                           0.0254                         0.5757  
Low                            0.0199                         0.5584  

=== Disparities (across groups) ===
                               Gap (max-min)  Ratio (min/max)
Statistical Parity (DP)               0.0109            0.767
Equal Opportunity (TPR)               0.0243            0.802
Equalized Odds (FPR)                  0.0055            0.782
Predictive Parity (Precision)         0.0174            0.970

=== Conditional Statistical Parity ===
Weighted gap (max-